In [ ]:
# Importar bibliotecas necesarias
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import files

# Cargar el archivo .txt
uploaded = files.upload()

# Leer el archivo cargado
file_name = list(uploaded.keys())[0]
with open(file_name, 'r', encoding='utf-8') as file:
    text = file.read()

# Preprocesamiento del texto
tokenizer = Tokenizer(char_level=True, filters=None)  # Trabajamos a nivel de caracteres
tokenizer.fit_on_texts([text])
total_chars = len(tokenizer.word_index) + 1

# Crear secuencias para entrenamiento
seq_length = 40  # Secuencias cortas, adecuadas para datos limitados
step = 3
input_sequences = []
output_chars = []

for i in range(0, len(text) - seq_length, step):
    input_sequences.append(text[i: i + seq_length])
    output_chars.append(text[i + seq_length])

# Convertir texto a secuencias numéricas
input_sequences = tokenizer.texts_to_sequences(input_sequences)
output_chars = tokenizer.texts_to_sequences(output_chars)

# Convertir a arrays y categorizar la salida
X = np.array(input_sequences)
y = to_categorical(np.array(output_chars) - 1, num_classes=total_chars)

# Construir el modelo LSTM ajustado
model = Sequential([
    Embedding(total_chars, 64, input_length=seq_length),  # Embedding reducido
    LSTM(150, return_sequences=True),  # Primera capa LSTM
    Dropout(0.2),
    LSTM(150, return_sequences=True),  # Segunda capa LSTM
    Dropout(0.2),
    LSTM(150, return_sequences=False),  # Tercera capa LSTM
    Dropout(0.3),
    Dense(256, activation='relu'),  # Capa totalmente conectada
    Dense(total_chars, activation='softmax')  # Salida
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Entrenar el modelo
model.fit(X, y, batch_size=16, epochs=200)  # Más épocas para compensar datos pequeños

# Guardar el modelo
model.save('modelo_LSTM_pequeno_dataset.h5')

# Función para muestreo con temperatura
def sample_with_temperature(predictions, temperature=0.8):
    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions + 1e-8) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

# Función para generar texto con prefijo
def generate_text_with_prefix(prefix, max_words=50, temperature=0.8):
    result = prefix
    seed_text = prefix[-seq_length:]  # Limitar al tamaño del modelo
    word_count = len(prefix.split())

    while word_count < max_words:
        encoded = tokenizer.texts_to_sequences([seed_text])
        encoded = pad_sequences(encoded, maxlen=seq_length, truncating='pre')
        predictions = model.predict(encoded, verbose=0)[0]
        predicted_index = sample_with_temperature(predictions, temperature)
        predicted_char = tokenizer.index_word.get(predicted_index + 1, '')

        result += predicted_char
        seed_text = seed_text[1:] + predicted_char
        word_count = len(result.split())

        if predicted_char == ' ' and word_count >= max_words:
            break

    return result.strip()

# Modo conversación
def chat_with_model():
    print("Inicia una conversación con el modelo. Escribe 'salir' para terminar.")
    while True:
        prompt = input("Tú: ")
        if prompt.lower() == 'salir':
            break
        response = generate_text_with_prefix(prompt, max_words=50, temperature=0.8)
        print(f"Modelo: {response}")

# Iniciar conversación
chat_with_model()


Saving intervencionesAbascal.txt to intervencionesAbascal.txt
Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


470/470 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 3.1124
Epoch 2/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 2.9421
Epoch 3/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 2.7248
Epoch 4/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 2.4629
Epoch 5/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 2.2992
Epoch 6/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 2.2170
Epoch 7/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 2.1380
Epoch 8/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 2.0850
Epoch 9/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 2.0617
Epoch 10/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 2.0061
Epoch 11/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 1.9259
Epoch 12/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 1.8477
Epoch 13/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 1.8138
Epoch 14/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 1.7822
Epoch 15/200
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step 

Inicia una conversación con el modelo. Escribe 'salir' para terminar.
Tú: Sánchez dice usted que
Modelo: Sánchez dice usted que han mentiras señor sánchez, ¿qué nos casor es una ay gobierno paren abaña. ló que nos aher, en el tranción socialistas que es— el importanes de exceño, señor sánchez, ¿qué nos ahuca que saburas que no los pageras de diviliy libertades farmo es por señor s
Tú: Además, Casado, su partido
Modelo: Además, Casado, su partido y el señor á0s, en el infertades y de proporto, que no pagerte para lo que no seson una afoccionel. la fayaba se a el decordos y libertades farmo es pasa, qué vampién a el gobierno paren abaña, permito nuevo, pero ¿vabierno pero ¿ve calle p
Tú: Sánchez únicamente subes impuestos
Modelo: Sánchez únicamente subes impuestos sinchez, ha dicepresidentaria de esto cámara. gorque no para seguras y que las ahabar a la úutaminto tedién han cenjierno parque no paren la factura de la luionel. permito ni a sus mentiras. preside esta cámara. gamo la crimina

KeyboardInterrupt: 